In [1]:
import spacy
import pickle
import random


In [2]:
train_data = pickle.load(open('train_data.pkl', 'rb'))

In [3]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [4]:
nlp = spacy.blank('en')

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:

        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner,last= True)
    
      
       
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(15):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)

In [5]:
train_model(train_data)

Statring iteration 0
{'ner': 11825.493416003843}
Statring iteration 1
{'ner': 8877.337969233731}
Statring iteration 2
{'ner': 9237.962029556185}
Statring iteration 3
{'ner': 6369.722762417509}
Statring iteration 4
{'ner': 6779.813200101318}
Statring iteration 5
{'ner': 6222.12232022646}
Statring iteration 6
{'ner': 5527.618657266225}
Statring iteration 7
{'ner': 5433.264416887634}
Statring iteration 8
{'ner': 5483.377653225647}
Statring iteration 9
{'ner': 4195.861026282969}
Statring iteration 10
{'ner': 5553.306237110091}
Statring iteration 11
{'ner': 4184.547270233433}
Statring iteration 12
{'ner': 3912.6131289720156}
Statring iteration 13
{'ner': 4092.188635647085}
Statring iteration 14
{'ner': 3620.751422809575}


In [6]:
nlp.to_disk('nlp_model')

In [7]:
nlp_model = spacy.load('nlp_model')

In [8]:
train_data[0][0]

'Vijayalakshmi Govindarajan SAP as a Consultant - SAP Basis  Chennai, Tamil Nadu - Email me on Indeed: indeed.com/r/Vijayalakshmi-Govindarajan/ d71bfb70a66b0046  Willing to relocate: Anywhere  WORK EXPERIENCE  SAP as a Consultant  SAP Basis -  May 2012 to Present  Involved in 3 support Projects and 1 implementation Project. • System Build, System Refresh, system upgrade and system maintenance activities. • Self-motivated and goal-oriented, with a strong ability to organize and prioritize tasks efficiently. • Excellent team player with good communication, analytical, problem solving skills. • Certification Done: Oracle certified Java Programmer • Finished Diploma in Java course from NIIT  EDUCATION  MCA in Computer Applications  Thiagarajar School of Management -  Madurai, Tamil Nadu  BSc  Sri Sathya Sai Institute of Higher Learning -  Anantapur, Andhra Pradesh  HSC  TVS Lakshmi Matriculation Higher Secondary School -  Madurai, Tamil Nadu  SSLC  TVS Lakshmi Matriculation Higher Secondar

In [9]:
import re
pattren_ex = r'\s\d?\d\s'
pattren_email = r'\s[A-Z]+@[A-Z]+.[A-Z]+.|:\s[a-z]+@[a-z]+.[a-z]+.'
text=""
def re_experience(text):
    expert =re.search(pattren_ex ,text)
    text =expert.group(0)
    return text
def re_email(text):
    expert =re.search(pattren_email ,text)
    text =expert.group(0)
    return text    

In [10]:
import sys, fitz
data=[]
skills=[]
email=[] 
name=[] 
degree=[]
experience=[]
PATH=""
numberCv=0
def set_path(path):
    PATH = path
    return  PATH
def get_path():
    return  PATH
def number_cv(number_Cv):
    numberCv = number_Cv
    return  numberCv
def read_CVs(PATH,numberCv):
    n='0'
  
    for  n in range(number_cv(numberCv)):
        t = int(n)
        v = str(t)
        fname = set_path(PATH)+v+'.pdf'
        
        doc = fitz.open(fname)
        text = ""
     
        for page in doc:
            text = text + str(page.getText())

            tx = " ".join(text.split('\n'))
           # print("************",tx)
            doc.close
        re_experience(tx) 
        experience.append(re_experience(tx))
        re_email(tx)
        email.append(re_email(tx))
        doc = nlp_model(tx)
        for ent in doc.ents:
            print(f'{ent.label_.upper():{30}}- {ent.text}\n\n\n')
            data.append(ent.label_.upper()+'          '+ent.text)
           # print('************************',data[ent])
    for  n in range(len(data)):        
        print(data[n])
    for  i in range(len(data)):
        DEGREE = data[i].find('DEGREE')
        print ("SKILLES 22222222222222222222222222222222222222   ", DEGREE)
        result = data[i].find('SKILLS')
        NAME = data[i].find('NAME')
        EMAIL_ADDRESS = data[i].find('EMAIL ADDRESS')
       
        if result == 0 :
            print ("SKILLES", result ,'      ',i,data[i])
            skills.append(data[i])
        if NAME == 0 :
            print ("NAME        ", result ,'      ',i,data[i])
            name.append(data[i]) 
        #if EMAIL_ADDRESS== 0:   
            #print ("EMAIL_ADDRESS        ", result ,'      ',i,data[i])
            #email.append(data[i]) 
        if DEGREE >= 0:   
            print ("DEGREE         ", result ,'      ',i,data[i])
            degree.append(data[i]) 
    print('************************',len(data))
    

In [11]:
read_CVs('C:\\Users\\hp\\Desktop\\project2cv\\cv\\cv',6)
get_path()
i=0
print('*********AAAAAAAAAAA***************')
print('*********AAAAAAAAAAA***************')
for i in range(len(skills)):        
    print(skills[i])
for i in range(len(name)):        
    print(name[i])
for i in range(len(email)):        
    print(email[i])
for i in range(len(experience)):        
    print(experience[i])
print(len(experience))


NAME                          - Salma alsalam



LOCATION                      - Dooma



EMAIL ADDRESS                 - salmaal@gmail.com-



DEGREE                        - BSc



COLLEGE NAME                  - DAMASCUS



GRADUATION YEAR               - 2004



NAME                          - Bayan Nakresh



LOCATION                      - Domair



EMAIL ADDRESS                 - bayannk@gmail.com-



DESIGNATION                   - Software developer



DEGREE                        - BSc DAMASCUS



SKILLS                        - C++, Html, JavaScript, Software Development, MachineLearning, Natural Language Processing, word,



NAME                          - Aya nazha



LOCATION                      - Zamalka



COMPANIES WORKED AT           - Microsoft



YEARS OF EXPERIENCE           - 6 years



DESIGNATION                   - Software developer



DEGREE                        - STUDENT



SKILLS                        - C++, Html, Unity, Programming,Speaking English,



In [12]:
job =[]
print(len(skills))
def enter_job_skill(str):
    job.append(str) 
print(len(job))        

5
0


In [13]:
str=input("the num job ")
enter_job_skill(str)
str=input("the num job ")
enter_job_skill(str)
str=input("the num job ")
enter_job_skill(str)
def convert_lower():
    i=0
    for i in range(len(skills)):        
        if skills [i].islower()==False:
            skills[i]=skills[i].lower()
            print(skills[i])
    for i in range(len(job)):        
        if job [i].islower()==False:
            job [i]=job [i].lower()
            print(job [i])
    for i in range(len(degree)):        
        if degree [i].islower()==False:
            degree [i]=degree [i].lower()
            print(degree [i])        
   # for i in range(len(degree)):        
      #  if degree [i].islower()==False:
           # degree [i]=degree [i].lower()
           # print(degree [i])         
#convert_lower() 
print(job [len(job)-1])

the num job html
the num job Sql
the num job php
php


In [14]:
print(job)


['html', 'Sql', 'php']


In [17]:
accept_percent=[]
print(len(skills)) 
accept_percent = [0 for i in range(len(email))] 
print(accept_percent)
print(job)
def rankncing_skilles():
    i =0
    j =0
    convert_lower() 
    for i in range(len(job)): 
        print("**************")
        for j in range(len(skills)): 
            result = skills[j].find(job [i])
            if result>=0:
                print("pre  ",accept_percent[j])
                accept_percent[j]+=1
                print("aftr ",accept_percent[j])
            print(result)

def rankncing_degree(Degree):
    i =0
    j =0
    convert_lower()
    if Degree.islower()==False:
        print(Degree)
        Degree=Degree.lower()
    print(degree)
    print("len :",len(degree))
    for j in range(len(degree)): 
        print("pre@@@@@@####",degree[j])
        result = degree[j].find(Degree)
        if result>=0:
            print("pre  ",accept_percent[j])
            accept_percent[j]+=1
            print("aftr ",accept_percent[j])
        print(result)  
def rankncing_experience (exper):
    i =0
    j =0
    convert_lower()
    if exper.islower()==False:
        exper=exper.lower()
    print(experience)
    print("@@@@@@",len(experience))
    for j in range(len(experience)): 
        print("pre@@@@@@####",experience[j])
        result = experience[j].find(exper)
        if result>=0:
            print("pre  ",accept_percent[j])
            accept_percent[j]+=1
            print("aftr ",accept_percent[j])
        print(result)          

5
[0, 0, 0, 0, 0, 0]
['html', 'sql', 'php']


In [18]:
print(job[len(job )-1] ) 
rankncing_skilles()
print(accept_percent)
deg=input("the your degree ")
rankncing_degree(deg)
print(accept_percent) 
deg=input("the your experience ")
rankncing_experience (deg)
print(accept_percent) 

php
**************
pre   0
aftr  1
21
pre   0
aftr  1
21
pre   0
aftr  1
21
pre   0
aftr  1
16
pre   0
aftr  1
21
**************
-1
-1
-1
pre   1
aftr  2
41
-1
**************
-1
-1
pre   1
aftr  2
26
pre   2
aftr  3
84
pre   1
aftr  2
39
[1, 1, 2, 3, 2, 0]
the your degree bsc
['degree          bsc', 'degree          bsc damascus', 'degree          student', 'degree          bsc', 'degree          bsc', 'degree          bsc']
len : 6
pre@@@@@@#### degree          bsc
pre   1
aftr  2
16
pre@@@@@@#### degree          bsc damascus
pre   1
aftr  2
16
pre@@@@@@#### degree          student
-1
pre@@@@@@#### degree          bsc
pre   3
aftr  4
16
pre@@@@@@#### degree          bsc
pre   2
aftr  3
16
pre@@@@@@#### degree          bsc
pre   0
aftr  1
16
[2, 2, 2, 4, 3, 1]
the your experience 4
[' 2 ', ' 5 ', ' 6 ', ' 4 ', ' 7 ', ' 4 ']
@@@@@@ 6
pre@@@@@@####  2 
-1
pre@@@@@@####  5 
-1
pre@@@@@@####  6 
-1
pre@@@@@@####  4 
pre   4
aftr  5
1
pre@@@@@@####  7 
-1
pre@@@@@@####  4 
pre   1
aftr  2
1

In [27]:
def optimal_cv():
    i=0
    percent= accept_percent[0]
    for i in range(len(accept_percent)):
        if percent<=accept_percent[i]:
            percent=accept_percent[i]
            j=i
    print("accept_percent:",(accept_percent[j]*100)/(len(job )+2),name[j],email[j],"cv",j )      
    
            
        
    

In [28]:
optimal_cv()

accept_percent: 100.0 NAME          Alaa Alnablsi : alaana@gmail.com- cv 3
